# PQTrees
#### Lev. G 2020

### Project mostly based on the work described at:
* __Finding All Common Intervals of k Permutations:__ `docs/articles/Finding All Common Intervals of k Permutations.pdf`

* __Gene Proximity Analysis Across Whole Genomes via PQ Trees__: `docs/articles/Gene Proximity Analysis Across Whole Genomes via PQ Trees (2005) OrenWeimann.pdf`

## The project:
### This notebook is devided into a few subpartts as follows:

i. Common intervals   
ii. PQTrees    
iii. PQTrees extensions    
iv. Apendix   

Each part will provide an explanatiion, an implementation and further remarks

## Common Intervals 

### Glossary

#### 1. Common Intervals

For `k` permutation over the same alephabet, w.l.o.g over `[0..x]` and first permutation `p0` is `[0..x]`,
Common intervals are sub intervals of `p0` (for example `[1..3]`) tha appear together in all the permutation.

Trivial common intervals are all the singltons (because the permutations are over the same alphabet) - 
`[0,0]` ... `[x,x]`,   
and the whole permutation `[0,x]`

__Example:__   
For permutations `(0, 1, 2, 3, 4) ; 4, 3, 0, 2, 1)` the non trivial common intervals are:
`{ [1,2], [3,4], [0,2], [0,3] }`

#### 2. Irreducible intervals 
For `k` permutation as in `1.` irreducible intervals are a sub set of the common intervals set, 
that follow:  

Each interval that is not composable from any other intervals

__Composoble:__ An interval is said to be composble in the case there are some other intervals that overlap with at leasr 1 character and compose the said interval, for ex. composing `[2,4]` with `[3,5]` produces `[2,5]`

__Example:__ For athe following group of intervals:   
`{[1, 2], [1, 3], [1, 8], [1, 9], [2, 3], [4, 5], [4, 6], [4, 7], [4, 8], [4, 9], [5, 6] }`

The subset of irreducible intervlas is:   
`{[1, 2], [1, 8], [2, 3], [4, 5], [4, 7], [4, 8], [4, 9], [5, 6]}`


### Algorithms

In order to find all the common intervals, first a trivial algorithm was used - then amore eficient one was devised.

#### 1. Trivial algorithm

```python
for size in [1, ..., len(permutations[0])]:
    for w in sliding_window(permutations[0], size):
        char_set = set(w)
        if all(find_char_set(p, char_set) for p in permutations[1:]):
            common_intervals.add(w)

```

In [1]:
from pqtrees import trivial_common_k

permutations =  ((0, 1, 2, 3, 4), (4, 3, 0, 2, 1))
trivial_common_k(*permutations)

[CI[1, 2], CI[3, 4], CI[0, 2], CI[0, 3], CI[0, 4]]

#### 2. Optimized Algorithm

The optimized algorithm before searching a char_set in some pemutation - will index all the subsets of such permutation with the length of char_set - reducing the runtime of subsequent searched to `O(1)`.

Index generation is done lazily (the index is updated only upon first search of a certain length), to eliminate not needed proccessing when working with big anounts of permutations (more than 20-30).

The algorithm structure is very similar to `1.` but with indexing changes:

```python
for size in [1, ..., len(permutations[0])]:
    index = {}
    for w in sliding_window(permutations[0], size):
        char_set = set(w)
        if char_set_in_others(permutations[1:], char_set, index):
            common_intervals.add(w)

def char_set_in_others(others, char_set, index):
    for other in others:
        if other not in index:
            index_perm(index, other, len(char_set)
        if char_set not in index[other]:
            return False
    return True
                                      
def index_perm(index, perm, length):
    for w in sliding_window(perm, length):
        index[perm].add(set(w))
```

In [2]:
from pqtrees import common_k_indexed

permutations =  ((0, 1, 2, 3, 4), (4, 3, 0, 2, 1))
common_k_indexed(*permutations)

[CI[1, 2], CI[3, 4], CI[0, 2], CI[0, 3], CI[0, 4]]

#### 3. Benchmark
We want to measure the improvement of the second alg.
Thus we will run randomized examples on a large amount of long permutations and measure the times.

In [9]:
import random
from pprint import pprint
from pqtrees import time_runtime

REPEAT_TEST_TIMES = 150
LENGTH = 100
NUM_PERMS = 100

algs = [trivial_common_k, common_k_indexed]
names = [alg.__name__ for alg in algs]
total_times = {name: 0 for name in names}


for _ in range(REPEAT_TEST_TIMES):

    sig_a = list(range(LENGTH))
    other_perms = [list(sig_a) for _ in range(NUM_PERMS - 1)]
    for p in other_perms:
        random.shuffle(p)

    t_others = map(tuple, other_perms)

    for alg, alg_name in zip(algs, names):
        _, cur_rt = time_runtime(lambda: alg(sig_a, *t_others))
        total_times[alg_name] += cur_rt

print("\nRuntimes:")
pprint(total_times)



Runtimes:
{'common_k_indexed': 0.9531488960001298, 'trivial_common_k': 54.91819818200008}


## PQTree
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6e/Pq-tree-5-leaves.svg/330px-Pq-tree-5-leaves.svg.png" alt="From Wikipedia" style="width: 200px;" align=”left” />

<img src=”IMAGE URL” align=”right” style=”margin: 0px 0px 0px 10px;” /><p>Your text goes here.</p>